If we create a script we can all with some output

In [1]:
$script = @"
ECHO 'A thing'
ECHO 'Another thing'
ECHO 'Yet another thing'
ECHO 'Maybe this is progress'
"@

$cmdScriptBlock = [scriptblock]::Create($script)
        Invoke-Command -ScriptBlock $cmdScriptBlock | Tee-Object -Variable r

A thing
Another thing
Yet another thing
Maybe this is progress


and then add it into a framework of your code

In [2]:
function Set-S3Object {
    $script = @"
ECHO 'A thing'
ECHO 'Another thing'
ECHO 'Yet another thing'
ECHO 'Maybe this is progress'
"@

    $cmdScriptBlock = [scriptblock]::Create($script)
    Invoke-Command -ScriptBlock $cmdScriptBlock | Tee-Object -Variable r  ## This output is sent to the variable
}

function Copy-GlacierToS3 {
    Param($file)
    $resultObj = [PSCustomObject]@{
        LocalFile = $file
        Status    = "always"
    }
    Set-S3Object
    $resultObj.Status = "Success"


    return $resultObj
}

# DO Migration
$resultList = @()
$filesToMigrate = 'file1','file2','file3'
# foreach, call Copy-GlacierToS3
foreach ($f in $filesToMigrate) {
    $res = Copy-GlacierToS3 -File $f
    
    $resultList += $res
    Start-Sleep -Seconds 2
}

# display results
$resultList # This is where output will occur

A thing
Another thing
Yet another thing
Maybe this is progress

LocalFile Status 
--------- ------ 
file1     Success
A thing
Another thing
Yet another thing
Maybe this is progress
file2     Success
A thing
Another thing
Yet another thing
Maybe this is progress
file3     Success




So you would have to do something clever with jobs to get the output - like this - Notebooks will give you the output at the end by default but if you run this code in the console you will see what is happening

In [13]:
$JobName = 'Jobb'
$JobScript = "Write-Output 'I am a job';Start-Sleep -Seconds 20 ; Write-Output 'I am further in';Start-Sleep -Seconds 20 ; Write-Output 'I have finished'"
Start-Job -Name $JobName {Write-Output 'I am a job';Start-Sleep -Seconds 20 ; Write-Output 'I am further in';Start-Sleep -Seconds 20 ; Write-Output 'I have finished'}
$job = Get-Job -Name $Jobname
While (($job.State -ne 'Completed') -and ($job.State -ne 'Failed')){
    $job = Get-Job -Name $Jobname
    $jobresult = Receive-Job -Name $Jobname
    $JobResult
    Start-Sleep -Seconds 10
}
$jobresult = Receive-Job -Name $Jobname
$JobResult
Remove-Job -Name $JobName -Force



Id     Name            PSJobTypeName   State         HasMoreData     Location             Command                  
--     ----            -------------   -----         -----------     --------             -------                  
19     Jobb            BackgroundJob   Running       True            localhost            Write-Output 'I am a j...
I am a job
I am further in
I have finished




BUt in this scenario it is really hard to do as the powershell cant read the output of the invoke-comand that is running the S3 upload command in real time as it sends the command off and doesnt get it back until the end

In [ ]:
function Set-S3Object {
    $script = @"
ECHO 'A thing'
ECHO 'Another thing'
ECHO 'Yet another thing'
ECHO 'Maybe this is progress'
"@

    $cmdScriptBlock = [scriptblock]::Create($script)
    Invoke-Command -ScriptBlock $cmdScriptBlock | Tee-Object -Variable r  ## This output is sent to the variable
}

function Copy-GlacierToS3 {
    Param($file)
    $resultObj = [PSCustomObject]@{
        LocalFile = $file
        Status    = "always"
    }
    Set-S3Object
    $resultObj.Status = "Success"


    return $resultObj
}

# Do Migration
$resultList = @()
$filesToMigrate = 'file1', 'file2', 'file3'
# foreach, call Copy-GlacierToS3
foreach ($f in $filesToMigrate) {
    $JobName = 'Jobb'
    Start-Job -Name $JobName {$res = Copy-GlacierToS3 -File $f }
    $job = Get-Job -Name $Jobname
    While (($job.State -ne 'Completed') -and ($job.State -ne 'Failed')) {
        $job = Get-Job -Name $Jobname
        $jobresult = Receive-Job -Name $Jobname
        $JobResult
        Start-Sleep -Seconds 10
    }
    $jobresult = Receive-Job -Name $Jobname
    $JobResult
    Remove-Job -Name $JobName -Force
    
    $resultList += $res
    Start-Sleep -Seconds 2
}

# display results
$resultList # This is where output will occur

In [11]:
$jobresult